In [3]:
import os
from dotenv import dotenv_values
import supervision as sv

os.environ["KMP_DUPLICATE_LIB_OK"]="TRUE"
config = dotenv_values("../.env") 

In [7]:
import numpy
numpy.__config__.show()
import scipy
scipy.show_config()

Build Dependencies:
  blas:
    detection method: pkgconfig
    found: true
    include directory: /c/opt/64/include
    lib directory: /c/opt/64/lib
    name: openblas64
    openblas configuration: USE_64BITINT=1 DYNAMIC_ARCH=1 DYNAMIC_OLDER= NO_CBLAS=
      NO_LAPACK= NO_LAPACKE= NO_AFFINITY=1 USE_OPENMP= SKYLAKEX MAX_THREADS=2
    pc file directory: C:/opt/64/lib/pkgconfig
    version: 0.3.23.dev
  lapack:
    detection method: internal
    found: true
    include directory: unknown
    lib directory: unknown
    name: dep1991176392224
    openblas configuration: unknown
    pc file directory: unknown
    version: 1.26.4
Compilers:
  c:
    commands: cl
    linker: link
    name: msvc
    version: 19.29.30153
  c++:
    commands: cl
    linker: link
    name: msvc
    version: 19.29.30153
  cython:
    commands: cython
    linker: cython
    name: cython
    version: 3.0.8
Machine Information:
  build:
    cpu: x86_64
    endian: little
    family: x86_64
    system: windows
  host:

In [ ]:
ds = sv.DetectionDataset.from_yolo(
    images_directory_path=f"dataset/train/images",
    annotations_directory_path=f"dataset/train/labels",
    data_yaml_path=f"dataset/data.yaml"
)